In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
from dateutil import parser
from py_vollib_vectorized.implied_volatility import vectorized_implied_volatility as iv

plt.style.use('classic')

In [2]:
df_quotes = pd.read_csv("../outs/deribit_quotes_history.csv.gz", compression='gzip')
df_quotes.head()

,Unnamed: 0,symbol,timestamp,local_timestamp,ask_amount,ask_price,bid_price,bid_amount
0,0,BTC-24SEP21-32000-P,1624665599536000,1624665600283655,0.5,0.1945,0.1915,1.3
1,1,BTC-24SEP21-32000-P,1624665623151000,1624665623175673,0.4,0.1960,0.1915,1.3
2,2,BTC-24SEP21-32000-P,1624665623286000,1624665623304450,1.4,0.1955,0.1915,1.3
3,3,BTC-24SEP21-32000-P,1624665623365000,1624665623380090,1.2,0.1955,0.1915,1.3
4,4,BTC-24SEP21-32000-P,1624665623577000,1624665623590435,1.0,0.1955,0.1915,1.3


In [3]:
# process quote
df_quotes['datetime'] = pd.to_datetime(df_quotes['timestamp'], unit='us')
df_quotes.sort_values(by="datetime", inplace=True, ascending=True)
df_quotes.head()

,Unnamed: 0,symbol,timestamp,local_timestamp,ask_amount,ask_price,bid_price,bid_amount,datetime
8306643,8306643,BTC-27DEC19-10000-P,1567900793710000,1567900800299199,14.8,0.1330,0.1255,8.5,2019-09-07 23:59:53.710
14683849,14683849,BTC-27DEC19-10000-C,1567900798349000,1567900800299208,7.0,0.1985,0.1920,18.4,2019-09-07 23:59:58.349
14683850,14683850,BTC-27DEC19-10000-C,1567900811779000,1567900811787923,7.0,0.1990,0.1920,18.4,2019-09-08 00:00:11.779
14683851,14683851,BTC-27DEC19-10000-C,1567900812323000,1567900812336936,7.0,0.1990,0.1920,22.7,2019-09-08 00:00:12.323
14683852,14683852,BTC-27DEC19-10000-C,1567900812335000,1567900812349516,11.3,0.1990,0.1920,22.7,2019-09-08 00:00:12.335


In [4]:
# set the datetime index on
df_quotes = df_quotes[['datetime', 'ask_amount', 'ask_price', 'bid_price', 'bid_amount', 'symbol']].set_index(['datetime'])
df_quotes.head()

,ask_amount,ask_price,bid_price,bid_amount,symbol
datetime,,,,,
2019-09-07 23:59:53.710,14.8,0.1330,0.1255,8.5,BTC-27DEC19-10000-P
2019-09-07 23:59:58.349,7.0,0.1985,0.1920,18.4,BTC-27DEC19-10000-C
2019-09-08 00:00:11.779,7.0,0.1990,0.1920,18.4,BTC-27DEC19-10000-C
2019-09-08 00:00:12.323,7.0,0.1990,0.1920,22.7,BTC-27DEC19-10000-C
2019-09-08 00:00:12.335,11.3,0.1990,0.1920,22.7,BTC-27DEC19-10000-C


In [5]:
# making a group
quote_group = df_quotes.groupby([pd.Grouper(freq='d'), 'symbol'])

In [6]:
# read 1m history charts
df_1m = pd.read_csv('../datasets/Binance_BTCUSDT_minute.csv', header=1,)
df_1m['datetime'] = pd.to_datetime(df_1m['unix'], unit='ms')
df_1m = df_1m.set_index('datetime')
df_1m.sort_values(by="datetime", inplace=True, ascending=True)

In [7]:
# merging 1m chart close price to df_quotes 
df_quotes = pd.merge_asof(df_quotes, df_1m['close'], on='datetime', direction='nearest', tolerance=pd.Timedelta(seconds=1000))
df_quotes

,datetime,ask_amount,ask_price,bid_price,bid_amount,symbol,close
0,2019-09-07 23:59:53.710,14.8,0.1330,0.1255,8.5,BTC-27DEC19-10000-P,NaN
1,2019-09-07 23:59:58.349,7.0,0.1985,0.1920,18.4,BTC-27DEC19-10000-C,NaN
2,2019-09-08 00:00:11.779,7.0,0.1990,0.1920,18.4,BTC-27DEC19-10000-C,NaN
3,2019-09-08 00:00:12.323,7.0,0.1990,0.1920,22.7,BTC-27DEC19-10000-C,NaN
4,2019-09-08 00:00:12.335,11.3,0.1990,0.1920,22.7,BTC-27DEC19-10000-C,NaN
...,...,...,...,...,...,...,...
29139848,2021-12-09 23:59:43.559,18.0,0.2150,0.2010,34.0,BTC-25MAR22-50000-P,NaN
29139849,2021-12-09 23:59:43.701,18.0,0.2150,0.2010,9.0,BTC-25MAR22-50000-P,NaN
29139850,2021-12-09 23:59:44.579,18.0,0.2150,0.2015,9.0,BTC-25MAR22-50000-P,NaN
29139851,2021-12-09 23:59:44.605,18.0,0.2150,0.2015,18.0,BTC-25MAR22-50000-P,NaN


In [8]:
df_quotes.dropna(inplace=True)
df_quotes.head()

,datetime,ask_amount,ask_price,bid_price,bid_amount,symbol,close
4432,2019-09-08 17:40:51.802,15.5,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0
4433,2019-09-08 17:41:22.521,11.0,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0
4434,2019-09-08 17:41:23.664,15.5,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0
4435,2019-09-08 17:41:53.201,16.0,0.1345,0.129,8.0,BTC-27DEC19-10000-P,10000.0
4436,2019-09-08 17:43:04.867,11.0,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0


In [9]:
def get_instrument_regex_groups(symbols, prefix='BTC'):
    return symbols.str.extract(f"^{prefix}-([0-9]{{1,2}}[A-Z]{{3}}[0-9]{{2}})-([0-9]*)-(P|C)$")

def get_delta_in_year(date_expiry, dt):
    dt_expiry = pd.to_datetime(date_expiry, format='%d%b%y')
    tdelta = dt_expiry - dt
    delta_days = tdelta.dt.days

    return delta_days / 365.0

In [10]:
unique_symbols = pd.DataFrame(df_quotes['symbol'].unique(), columns=['symbol'])
unique_symbols.head()

,symbol
0,BTC-27DEC19-10000-C
1,BTC-27DEC19-10000-P
2,BTC-27DEC19-9000-P
3,BTC-27DEC19-9000-C
4,BTC-27DEC19-8000-P


In [11]:
regex_groups = get_instrument_regex_groups(unique_symbols['symbol'])
regex_groups = regex_groups.rename(columns={0: 'date_expiry', 1: 'strike_price', 2: 'call_or_put'})
regex_groups.head()

,date_expiry,strike_price,call_or_put
0,27DEC19,10000,C
1,27DEC19,10000,P
2,27DEC19,9000,P
3,27DEC19,9000,C
4,27DEC19,8000,P


In [12]:
# concat two tables
unique_symbols = pd.concat([unique_symbols, regex_groups], axis=1)
unique_symbols.head()

,symbol,date_expiry,strike_price,call_or_put
0,BTC-27DEC19-10000-C,27DEC19,10000,C
1,BTC-27DEC19-10000-P,27DEC19,10000,P
2,BTC-27DEC19-9000-P,27DEC19,9000,P
3,BTC-27DEC19-9000-C,27DEC19,9000,C
4,BTC-27DEC19-8000-P,27DEC19,8000,P


In [13]:
df_quotes = pd.merge(df_quotes.assign(symbol=df_quotes.symbol.astype(str)), unique_symbols.assign(symbol=unique_symbols.symbol.astype(str)), how='left', on='symbol')

In [14]:
df_quotes.head()

,datetime,ask_amount,ask_price,bid_price,bid_amount,symbol,close,date_expiry,strike_price,call_or_put
0,2019-09-08 17:40:51.802,15.5,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000,C
1,2019-09-08 17:41:22.521,11.0,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000,C
2,2019-09-08 17:41:23.664,15.5,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000,C
3,2019-09-08 17:41:53.201,16.0,0.1345,0.129,8.0,BTC-27DEC19-10000-P,10000.0,27DEC19,10000,P
4,2019-09-08 17:43:04.867,11.0,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000,C


In [15]:
df_quotes['time_to_expiry'] = get_delta_in_year(df_quotes['date_expiry'], df_quotes['datetime'])
df_quotes.head()

,datetime,ask_amount,ask_price,bid_price,bid_amount,symbol,close,date_expiry,strike_price,call_or_put,time_to_expiry
0,2019-09-08 17:40:51.802,15.5,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000,C,0.29863
1,2019-09-08 17:41:22.521,11.0,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000,C,0.29863
2,2019-09-08 17:41:23.664,15.5,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000,C,0.29863
3,2019-09-08 17:41:53.201,16.0,0.1345,0.129,8.0,BTC-27DEC19-10000-P,10000.0,27DEC19,10000,P,0.29863
4,2019-09-08 17:43:04.867,11.0,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000,C,0.29863


In [16]:
df_quotes['strike_price'] = df_quotes['strike_price'].apply(np.float64)
df_quotes.head()

,datetime,ask_amount,ask_price,bid_price,bid_amount,symbol,close,date_expiry,strike_price,call_or_put,time_to_expiry
0,2019-09-08 17:40:51.802,15.5,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000.0,C,0.29863
1,2019-09-08 17:41:22.521,11.0,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000.0,C,0.29863
2,2019-09-08 17:41:23.664,15.5,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000.0,C,0.29863
3,2019-09-08 17:41:53.201,16.0,0.1345,0.129,8.0,BTC-27DEC19-10000-P,10000.0,27DEC19,10000.0,P,0.29863
4,2019-09-08 17:43:04.867,11.0,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000.0,C,0.29863


In [17]:
df_quotes['call_or_put'] = df_quotes['call_or_put'].str.lower()
df_quotes.head()

,datetime,ask_amount,ask_price,bid_price,bid_amount,symbol,close,date_expiry,strike_price,call_or_put,time_to_expiry
0,2019-09-08 17:40:51.802,15.5,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000.0,c,0.29863
1,2019-09-08 17:41:22.521,11.0,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000.0,c,0.29863
2,2019-09-08 17:41:23.664,15.5,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000.0,c,0.29863
3,2019-09-08 17:41:53.201,16.0,0.1345,0.129,8.0,BTC-27DEC19-10000-P,10000.0,27DEC19,10000.0,p,0.29863
4,2019-09-08 17:43:04.867,11.0,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000.0,c,0.29863


In [18]:
df_quotes['ask_iv'] = iv(df_quotes['ask_price']*df_quotes['close'], df_quotes['close'], df_quotes['strike_price'], df_quotes['time_to_expiry'], 0.02, df_quotes['call_or_put'])
df_quotes['bid_iv'] = iv(df_quotes['bid_price']*df_quotes['close'], df_quotes['close'], df_quotes['strike_price'], df_quotes['time_to_expiry'], 0.02, df_quotes['call_or_put'])
df_quotes.head()

/Users/wendychan/opt/anaconda3/lib/python3.9/site-packages/py_vollib_vectorized/implied_volatility.py:75: UserWarning: Found Above Maximum Price contracts at index [1121175, 1121180, 1121208, 1121210, 1121212, 1121214, 1121216, 1121218, 1121220, 1121222, 1121225, 1121227, 1186823, 1186826, 1186827, 2138572, 2187390, 2187392, 2211140, 2211142, 4483098, 4483117, 4483118, 4483119, 4483120, 4483121, 4483122, 4483539, 4483540, 4483541, 4484035, 4484037, 4484295, 4484296, 4484298, 4484299, 4484301, 4484920, 4484922, 4484923, 4485710, 4485712, 4485714, 4485716, 4485737, 4485740, 14506491, 14506492, 14506493, 14506496, 14506498, 14506500, 14506501, 14506502, 14506505, 14506506, 14506509, 14506510, 14506511, 14506514, 14506516, 14506518, 14506519, 14506521, 14506522, 14506523, 14506524, 14506526, 14506528, 14506529, 14506530, 14506532, 14506533, 14506536, 14506538, 14506540, 14506541, 14506543, 14506545, 14506546, 14506547, 14506549, 14506552, 14506555, 14506557, 14506558, 14506561, 14506562, 1

,datetime,ask_amount,ask_price,bid_price,bid_amount,symbol,close,date_expiry,strike_price,call_or_put,time_to_expiry,ask_iv,bid_iv
0,2019-09-08 17:40:51.802,15.5,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000.0,c,0.29863,0.882642,0.858976
1,2019-09-08 17:41:22.521,11.0,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000.0,c,0.29863,0.882642,0.858976
2,2019-09-08 17:41:23.664,15.5,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000.0,c,0.29863,0.882642,0.858976
3,2019-09-08 17:41:53.201,16.0,0.1345,0.129,8.0,BTC-27DEC19-10000-P,10000.0,27DEC19,10000.0,p,0.29863,0.635571,0.609895
4,2019-09-08 17:43:04.867,11.0,0.1930,0.188,8.0,BTC-27DEC19-10000-C,10000.0,27DEC19,10000.0,c,0.29863,0.882642,0.858976


In [19]:
df_quotes_with_iv = df_quotes[['datetime', 'ask_amount', 'ask_iv', 'bid_iv', 'bid_amount', 'symbol']].set_index(['datetime'])
df_quotes_with_iv.head()

,ask_amount,ask_iv,bid_iv,bid_amount,symbol
datetime,,,,,
2019-09-08 17:40:51.802,15.5,0.882642,0.858976,8.0,BTC-27DEC19-10000-C
2019-09-08 17:41:22.521,11.0,0.882642,0.858976,8.0,BTC-27DEC19-10000-C
2019-09-08 17:41:23.664,15.5,0.882642,0.858976,8.0,BTC-27DEC19-10000-C
2019-09-08 17:41:53.201,16.0,0.635571,0.609895,8.0,BTC-27DEC19-10000-P
2019-09-08 17:43:04.867,11.0,0.882642,0.858976,8.0,BTC-27DEC19-10000-C


In [20]:
# making a group
quote_group = df_quotes_with_iv.groupby([pd.Grouper(freq='d'), 'symbol'])

In [21]:
df_daily_quotes_sum = quote_group.sum()
df_daily_quotes_sum.head()

ask_amount       ask_iv       bid_iv  \
datetime   symbol                                                      
2019-09-08 BTC-27DEC19-10000-C      6491.3   430.743650   415.746492   
           BTC-27DEC19-10000-P      6518.6   450.829760   434.174268   
2019-09-09 BTC-27DEC19-10000-C     83828.0  4555.255639  4399.514935   
           BTC-27DEC19-10000-P     50688.7  5349.900455  5190.565884   
2019-09-10 BTC-27DEC19-10000-C    108248.2  5401.503624  5212.670372   

                                bid_amount  
datetime   symbol                           
2019-09-08 BTC-27DEC19-10000-C      8294.8  
           BTC-27DEC19-10000-P     12019.6  
2019-09-09 BTC-27DEC19-10000-C     89794.4  
           BTC-27DEC19-10000-P    125783.0  
2019-09-10 BTC-27DEC19-10000-C     96637.7

In [22]:
df_daily_option_price = pd.DataFrame(index=df_daily_quotes_sum.index)
df_daily_option_price['total_quote_amount'] = df_daily_quotes_sum['ask_amount'] + df_daily_quotes_sum['bid_amount']
df_daily_option_price['avg_iv'] = quote_group.apply(lambda x: np.average(x[['ask_iv', 'bid_iv']], weights=x[['ask_amount', 'bid_amount']]))
df_daily_option_price.head()

total_quote_amount    avg_iv
datetime   symbol                                           
2019-09-08 BTC-27DEC19-10000-C             14786.1  0.813613
           BTC-27DEC19-10000-P             18538.2  0.675237
2019-09-09 BTC-27DEC19-10000-C            173622.4  0.794151
           BTC-27DEC19-10000-P            176471.7  0.711817
2019-09-10 BTC-27DEC19-10000-C            204885.9  0.787959

In [23]:
df_daily_option_price = df_daily_option_price.reset_index([1])
df_daily_option_price.head()

,symbol,total_quote_amount,avg_iv
datetime,,,
2019-09-08,BTC-27DEC19-10000-C,14786.1,0.813613
2019-09-08,BTC-27DEC19-10000-P,18538.2,0.675237
2019-09-09,BTC-27DEC19-10000-C,173622.4,0.794151
2019-09-09,BTC-27DEC19-10000-P,176471.7,0.711817
2019-09-10,BTC-27DEC19-10000-C,204885.9,0.787959


In [24]:
quote_group_time = df_quotes[['symbol', 'time_to_expiry', 'datetime']].set_index('datetime').groupby([pd.Grouper(freq='d'), 'symbol'])
df_daily_quotes_time = quote_group_time.last()
df_daily_quotes_time.head()

time_to_expiry
datetime   symbol                             
2019-09-08 BTC-27DEC19-10000-C        0.298630
           BTC-27DEC19-10000-P        0.298630
2019-09-09 BTC-27DEC19-10000-C        0.295890
           BTC-27DEC19-10000-P        0.295890
2019-09-10 BTC-27DEC19-10000-C        0.293151

In [25]:
df_daily_quotes_time = df_daily_quotes_time.reset_index([1])
df_daily_quotes_time.head()

,symbol,time_to_expiry
datetime,,
2019-09-08,BTC-27DEC19-10000-C,0.298630
2019-09-08,BTC-27DEC19-10000-P,0.298630
2019-09-09,BTC-27DEC19-10000-C,0.295890
2019-09-09,BTC-27DEC19-10000-P,0.295890
2019-09-10,BTC-27DEC19-10000-C,0.293151


In [26]:
df_daily_option_price = pd.merge(df_daily_option_price.assign(symbol=df_daily_option_price.symbol.astype(str)), df_daily_quotes_time.assign(symbol=df_daily_quotes_time.symbol.astype(str)), on=['symbol', 'datetime'], how='left')
df_daily_option_price.head()

,symbol,total_quote_amount,avg_iv,time_to_expiry
datetime,,,,
2019-09-08,BTC-27DEC19-10000-C,14786.1,0.813613,0.298630
2019-09-08,BTC-27DEC19-10000-P,18538.2,0.675237,0.298630
2019-09-09,BTC-27DEC19-10000-C,173622.4,0.794151,0.295890
2019-09-09,BTC-27DEC19-10000-P,176471.7,0.711817,0.295890
2019-09-10,BTC-27DEC19-10000-C,204885.9,0.787959,0.293151


In [27]:
df_daily_option_price.to_csv('../outs/deribit_daily_avg_iv.csv')